In [9]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool  = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [16]:
from openai.types.chat.chat_completion import ChatCompletion

def chat_completion(prompt: str) -> object:
    response : ChatCompletion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="gpt-3.5-turbo"
    )
    display(dict(response))
    # print(response)
    return response.choices[0].message.content

print(chat_completion("What is 11+1?"))

{'id': 'chatcmpl-8phOiuVozJGJRl4PskkbIM0L1xDQM',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='11 + 1 = 12', role='assistant', function_call=None, tool_calls=None))],
 'created': 1707332240,
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21)}

11 + 1 = 12


In [15]:
def chat_completion(prompt: str) -> str:
    completion : ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=40,
    )
    # print(response)
    return completion.choices[0].message.content

print(chat_completion("Compose a poem that explains the concept of recursion in programming."))

In the land of loops and functions we dwell,
Where patterns of logic and sequences swell.
But a concept more mystical, beyond comprehension,
Is the beauty of recursive intervention.

Like a mirror reflecting its own


In [4]:
stream = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    messages = [{"role": "user", "content": "say this is a test"}],
    stream = True,
    
)

for part in stream:
    print(part.choices[0].delta.content or "")


This
 is
 a
 test
.



## JSON Mode

In [22]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    messages = [
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "List of months that have 30 days."}
    ],
)

print(response.choices[0].message.content)

{
  "months_with_30_days": [
    "April",
    "June",
    "September",
    "November"
  ]
}


In [27]:
import json
from typing import Dict

# Parse response
obj: Dict[str, list[str]] = json.loads(response.choices[0].message.content)

# The result is python Dictionary:
print(obj['months_with_30_days'])

NameError: name 'response' is not defined

In [5]:
def get_current_weather(location: str, unit: str= "fahrenheit")-> str: 
    if "tokyo" in location.lower():
        return json.dumps({"location": "tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "san francisco", "temperature": "50", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "10", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [2]:
from openai.types.chat.chat_completion import ChatCompletion
import json

def run_conversation(main_request: str) -> object:
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The City and state, e.g. San Francisco, CA"
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message: ChatCompletion = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Response Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function 
    if tool_calls:
        # Step 3: Call the function 
        # note: The JSON response may not always be valid; be sure to handle errors  
        available_functions = {
            "get_current_weather": get_current_weather
        } # only one function in this example, but you can have multiple

        messages.append(response_message) # extend conversation with assisstant's reply

        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls :
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )# extend conversation with function response
        display("* Second Request Messages: ", list(messages))   
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        ) # get a new response 
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [15]:
run_conversation("What is the weather like in San Francisco, Tokyo, and Paris?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_L1QuQWD9SQpDWKMvjhllzq0j', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_9nD0ketC8vmLJInuLUHCKB8a', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_laksRa2eRU7OncB3WJhyOKrp', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Response Tool Calls: '

[ChatCompletionMessageToolCall(id='call_L1QuQWD9SQpDWKMvjhllzq0j', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_9nD0ketC8vmLJInuLUHCKB8a', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_laksRa2eRU7OncB3WJhyOKrp', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': 'What is the weather like in San Francisco, Tokyo, and Paris?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_L1QuQWD9SQpDWKMvjhllzq0j', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_9nD0ketC8vmLJInuLUHCKB8a', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_laksRa2eRU7OncB3WJhyOKrp', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_L1QuQWD9SQpDWKMvjhllzq0j',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "san francisco", "temperature": "50", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_9nD0ketC8vmLJInuL

* Second Response:  {'id': 'chatcmpl-8pgAZLIO6GpRLz2wULRp5FICwVkYB', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, the weather in San Francisco is 50°F, in Tokyo it is 10°C, and in Paris it is also 10°C.', role='assistant', function_call=None, tool_calls=None))], 'created': 1707327519, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_04f9a1eebf', 'usage': CompletionUsage(completion_tokens=30, prompt_tokens=176, total_tokens=206)}


'Currently, the weather in San Francisco is 50°F, in Tokyo it is 10°C, and in Paris it is also 10°C.'

In [3]:
def order_function(order_name: str) -> str:
    if order_name == "pizza":
        return json.dumps({"order": "pizza", "status": "confirmed"})
    elif order_name == "burger":
        return json.dumps({"order": "burger", "status": "confirmed"})
    else:
        return json.dumps({"order": order_name, "status": "unknown"})
    
def run_conversation(main_request: str) -> object:
    messages: list = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "order_function",
                "description": "Place an order for food",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "order_name": {
                            "type": "string",
                            "description": "The name of the food to order, e.g. pizza, burger"
                        },
                    },
                    "required": ["order_name"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    response_message: ChatCompletion = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Response Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function 
    if tool_calls:
        # Step 3: Call the function 
        # note: The JSON response may not always be valid; be sure to handle errors  
        available_functions = {
            "order_function": order_function
        } # only one function in this example, but you can have multiple

        messages.append(response_message) # extend conversation with assisstant's reply

        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls :
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                order_name=function_args.get("order_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )# extend conversation with function response
        display("* Second Request Messages: ", list(messages))   
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        ) # get a new response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [4]:
run_conversation("Pizza order karna ha")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_xASOClgPMlKFIGaMO3hr7zfv', function=Function(arguments='{"order_name": "pizza"}', name='order_function'), type='function')]}

'* First Response Tool Calls: '

[ChatCompletionMessageToolCall(id='call_xASOClgPMlKFIGaMO3hr7zfv', function=Function(arguments='{"order_name": "pizza"}', name='order_function'), type='function')]

'* Second Request Messages: '

[{'role': 'user', 'content': 'Pizza order karna ha'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xASOClgPMlKFIGaMO3hr7zfv', function=Function(arguments='{"order_name": "pizza"}', name='order_function'), type='function')]),
 {'tool_call_id': 'call_xASOClgPMlKFIGaMO3hr7zfv',
  'role': 'tool',
  'name': 'order_function',
  'content': '{"order": "pizza", "status": "confirmed"}'}]

* Second Response:  {'id': 'chatcmpl-8phAq9xeGbZDcEkyt1dFVeJZDDLfU', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Pizza order successfully placed!', role='assistant', function_call=None, tool_calls=None))], 'created': 1707331380, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_04f9a1eebf', 'usage': CompletionUsage(completion_tokens=5, prompt_tokens=47, total_tokens=52)}


'Pizza order successfully placed!'